# Math 134A Week 4

In [2]:
import math
from scipy.optimize import fsolve
import numpy_financial as npf
import pandas as pd

# Computing the present value of a cash flow with interest rate r. 
def PV(r, cf, continuous_compounding = False, dt = None): 
    if dt is None: dt = range(len(cf))
    pv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            pv += cf[k] / (1+r)**dt[k]
    else:
        for k in range(0, len(cf)):
            pv += cf[k] * math.exp(-r * dt[k])
    return pv

# This will return the future value of a cash flow at time T. If T is not specified, then we take T as the time that we receive the last cash flow
def FV(r, cf, continuous_compounding = False, dt = None, T = None):
    if dt is None: dt = range(len(cf))
    if T is None: T = dt[-1]
    fv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            fv += cf[k] * (1+r)**(T - dt[k])
    else:
        for k in range(0, len(cf)):
            fv += cf[k] * math.exp(r * (T - dt[k]))
    return fv   

# Computing the internal rate of return by using the fsolve function to find roots to a polynomial
def irr(cf):
    def simple_npv(r, cf):
        return sum(cf[k]/(1+r)**k for k in range(len(cf)))
    
    result = fsolve(simple_npv, x0 = 0.1, args = cf)
    return result[0]

# Computing the internal rate of return by using the irr function in numpy financial library
def irr2(cf):
    return npf.irr(cf)

# This function computes the YTM of a bond mature in dt years, with face value F, price P, and coupon c paid m times every year
def YTM(F, P, dt, c = 0, m = 1):
    cf = [-P] + [c/m] * (dt * m - 1) + [F+c/m]
    return irr(cf) * m
    
# This function computes the price of a bond mature in dt years, with face value F, YTM y, and yearly coupon c paid m times every year 
def bond_price(F, y, dt, c = 0, m = 1):
    cf = [0] + [c/m] * (dt * m - 1) + [F+c/m]
    return PV(y/m, cf)

# This function will return the per period payment A if P is loaned for n periods, with interest r per period.
def A(P, r, n):
    return (P * r)/(1 - 1/(1+r)**n)

def macaulay_duration(c, y, m, n):
    """
    c is the coupon rate per period
    y is the yield per period
    m is the number of periods per year
    n is the number of periods remaining
    """
    D = (1+y)/(m*y) - (1 + y + n*(c-y))/(m*c * ((1+y)**n - 1) + m*y)
    return D

# Duration of a Portfolio
Assume that a portfolio consists of two bonds with payment cash flow $(0, c_1, c_2,\ldots, c_n)$ and $(0, d_1,\ldots, d_n)$. We may extend one of the cash flow with zeros so that they have the same length. We want to compute the duration of this portfolio. Let $P^A_k$ be the present value of the payment $c_k$, and $P^B_k$ be the resent value of the payment $d_k$. Let $P^A$ be the price/present value of the first bond, and $P^B$ be the price of the second bond. Let $D^A$ and $D^B$ to be their durations. Then we have 
$$D^A = \frac{\sum_1^n t_kP^A_k}{P^A}$$
$$D^B = \frac{\sum_1^n t_kP^B_k}{P^B}$$

Let $D$ be the duration of this portfolio, and $P = P_A+P_B$ to be the present value of this portfolio. Then we have 
$$D = \frac{\sum_1^n t_k (P^A_k+P^B_k)}{P} = \frac{D^AP^A+D^BP^B}{P} = \frac{P^A}{P}D^A + \frac{P^B}{P}D^B$$

In other words, the duration of a portfolio is a weighted sum of the duration of each individual bonds, and the weight is the proportion of this bonds price over the price of this portfolio.

More generally, if a portfolio consists of $n$ fixed income investments with price $P_1,\ldots, P_n$, and each of them has a duration $D_1,\ldots, D_n$. Let $P = P_1+\cdots+P_n$ be the price of this portfolio, and $D$ be its duration. Then we have 
$$D = \frac{P_1}{P}D_1+\cdots+\frac{P_n}{P}D$$

# Immunization
The goal of **Immunization** is to protect a portfolio against the change in the interest rate. If we have an obligation of paying $F$ amount of money in $D$ years. Suppose that the prevailing interest is $r$. Then the YTM of a bond will be close to $r$. A zero coupon bond with face value $F$ and mature in $D$ years will serve the purpose. However, it is very common that there is no such a bond in the market. We may construct a portfolio of other bonds so that the present value of this portfolio will equal to the present value of the future payment. However, if the interest rate change, then our portfolio may not worth the same as the payment that we need to deliver. 

Then what we do is to construct a portfolio consisting of fixed income securities so that its present value matches with the present value of the future payment, its duration matches with $D$, which is the time we need to deliver the payment, or equivalently, the duration of the hypothetical zero-coupon bond which mature in $D$ years. In this case, if the interest rate change, then the present value of the future payment and the present value of our portfolio will have the same amount of change. 

**Exercise 3.16:** (Bond selection) 
Consider the four bonds having annual payments as shown in the following table. They are traded to produce a 15\% yield. 

a) Determine the price of each bond. \
b) Determine the duration of each bond (not the modified duration). \
c) Which bond is most sensitive to a change in yield? \
d) Suppose you owe 2,000 at the end of 2 years. Concern about interest rate risk suggests that a portfolio consisting of the bonds and the obligation should be immunized. If
$V_A, V_B, V_C$, and $V_D$ are the total values of bonds purchased of types A,B,C, and D, respectively, what are the necessary constraints to implement the immunization? \
e) In order to immunize the portfolio, you decide to use bond C and one other bond. Which other bond should you choose? Find the amounts (in total value) of each of these to purchase. \
f) You decided in 5 to use bond C in the immunization. Would other choices, including perhaps a combination of bonds, lead to lower total cost?

<center>

|        | Bond A | Bond B | Bond C | Bond D |
|--------|--------|--------|--------|--------|
| Year 1 | 100    | 50     | 0      | 1000   |
| Year 2 | 100    | 50     | 0      | 0      |
| Year 3 | 1100   | 1050   | 1000   | 0      |

</center>

In [3]:
PV_a = PV(r = 0.15, cf = [0, 100, 100, 1100])
PV_b = PV(r = 0.15, cf = [0, 50, 50, 1050])
PV_c = PV(r = 0.15, cf = [0, 0, 0, 1000])
PV_d = PV(r = 0.15, cf = [0, 1000, 0, 0])
print(f"a) The price of each bonds are {PV_a: .2f}, {PV_b: .2f}, {PV_c:.2f}, {PV_d:.2f}")
d_a  = macaulay_duration(0.1, 0.15, 1, 3)
d_b  = macaulay_duration(0.05, 0.15, 1, 3)
d_c  = macaulay_duration(0, 0.15, 1, 3)
d_d  = macaulay_duration(0, 0.15, 1, 1)
print(f"b) The duration of each bonds are {d_a: .2f}, {d_b: .2f}, {d_c:.2f}, {d_d:.2f}")
print("c) The bond c is most sensitive to the change in yield")

a) The price of each bonds are  885.84,  771.68, 657.52, 869.57
b) The duration of each bonds are  2.72,  2.84, 3.00, 1.00
c) The bond c is most sensitive to the change in yield


d) Suppose that we bought $V_A,\ldots, V_D$ units of those $4$ bonds. Then the present value and duration of this portfolio is 
$$P = V_A \cdot 0.885 + V_B \cdot 0.771 + V_C \cdot 0.657 + V_D \cdot 0.869$$
$$D = \frac{V_A \cdot 0.885}{P} \cdot 2.72+\cdots + \frac{V_D \cdot 0.869}{P}\cdot 1$$
Then a necessary constrain to implement the immunization is that the following system of linear equations has at least a solution
$$\begin{cases}
\frac{2000}{1.15^2} = V_A \cdot 0.885 + V_B \cdot 0.771 + V_C \cdot 0.657 + V_D \cdot 0.869 \\
2 = \frac{V_A \cdot 0.885}{2000/1.15^2} \cdot 2.72+\cdots + \frac{V_D \cdot 0.869}{2000/1.15^2}\cdot 1 
\end{cases} $$

e) We may use bond C and bond D for computation. Then we have to solve the equation 
$$\begin{cases}
\frac{2000}{1.15^2} = V_C \cdot 0.657 + V_D \cdot 0.869 \\
2 = \frac{V_C \cdot 0.657}{2000/1.15^2} \cdot 3 + \frac{V_D \cdot 0.869}{2000/1.15^2}\cdot 1 
\end{cases} $$

By solving this equation, we get $V_C = 1150.9, V_D = 870.131$. In terms of their price, we need to spent $V_C \cdot 0.657 = 756.15$ to purchase bond C and $V_D\cdot 0.869 = 756.15$ to buy bond $D$. 

f) No, all choices will have the same price, which is the present value of the 2000 payment in 2 years. 

**Exercise 3.6**:
a) Consider a finite cashflow $(1, 3, 3^2,\ldots, 3^n)$. Suppose that the interest rate is $r$ and define $x = \frac{1}{1+r}$. Derive the present value $S_n$ of this cashflow in terms of $x$ and $x$. 
b) Consider an infinite cashflow $(1, 2, 3, 4, 5,\ldots)$. Suppose that the interest rate is $r$ and define $x = \frac{1}{1+r}$. Find the present value of this cash flow as a function of $r$ or $x$. 

**Solution:**
a)$$S_n = 1 + 3x + 3^2x^2+\cdots + 3^nx^n = \sum_0^n (3x)^k = \frac{(3x)^{n+1}-1}{3x-1}$$

b)$$PV = \sum_1^\infty kx^{k-1} = \frac{d}{dx} \sum_1^\infty x^k = \frac{d}{dx} \left(\frac{x}{1-x}\right) = \frac{1}{(1-x)^2}$$

**Exercise 3.10:** (Variable-rate mortgage)
The Smith family just took out a variable-rate mortgage on their new home. The mortgage value is 100,000, the term is 30 years, and initially the interest rate is 8\%. The interest rate is guaranteed for 5 years, after which time the rate will be adjusted according to prevailing rates. The new rate can be applied to their loan either by changing the payment amount or by changing the length of the mortgage.

(a) What is the original yearly mortgage payment? (Assume payments are yearly.)\
(b) What will be the mortgage balance after 5 years?\
(c) If the interest rate on the mortgage changes to 9% after 5 years, what will be the new yearly payment that keeps the termination time the same?\
(d) Under the interest change in (c), what will be the new term if the payments remain the same?

In [11]:
yearly_payment = A(100000, 0.08, 30)
print(f"a) The yearly payment is {yearly_payment:.2f}")
cf = [0] + [yearly_payment] * 25
R = PV(0.08, cf)
print(f"b) The remaining balance is {R:.2f}")
PV_remaining_balance = 100000 - PV(0.08, [0]+[yearly_payment] * 5)
remaining_balance = PV_remaining_balance * (1.08)**5
print(f"b) The remaining balance computed from the second method is also {remaining_balance:.2f}")
yearly_payment_new = A(remaining_balance, 0.09, 25)
print(f"c) The new yearly payment if the term remains the same is {yearly_payment_new:.2f}")
print(f"d) The new term if the yearly payment remain the same is {math.log(1 - remaining_balance * 0.09/yearly_payment, 1/1.09):.2f} years")

a) The yearly payment is 8882.74
b) The remaining balance is 94821.30
b) The remaining balance computed from the second method is also 94821.30
c) The new yearly payment if the term remains the same is 9653.40
d) The new term if the yearly payment remain the same is 37.57 years


# Annual Worth
Given a cash flow $(x_0,\ldots, x_n)$. The **annual worth** of this cashflow is the number $A$ such that $(0, A, \ldots, A)$ (ends in the n-th period) is an equivalent cashflow. The annual worth is useful for comparing investments with different time cycles.

**Exercise 3.2**: (Cycles and annual worth)
Given a cash flow stream $X = (x_0,x_1,x_2,...,x_n)$, a new stream $X_\infty$ of infinite length is made by successively repeating the corresponding finite stream. The interest rate is r. Let P and A be the present value and the annual worth, respectively, of stream X. Finally, let $P_\infty$ be the present value of stream $X_\infty$. Find A in terms of $P_\infty$ and conclude that $A$ can be used as well as $P_\infty$ for evaluation purposes.

**Solution:**
Let $P$ be the present value of the cash flow $X$. Then we have 
$$P_\infty = P+\frac{P}{(1+r)^{n+1}}+\frac{P}{(1+r)^{2(n+1)}}+\cdots = \frac{P}{1-\frac{1}{(1+r)^{n+1}}}$$
We also have 
$$P = \frac{A}{r}\left(1-\frac{1}{(1+r)^n}\right)$$
By substituting into the previous formula, we get 
$$P_\infty = \frac{A/r(1-\frac{1}{(1+r)^n})}{1-\frac{1}{(1+r)^{n+1}}}$$
Then we solve $A$ in terms of $P_\infty$, and we have 
$$A = \frac{P_\infty\cdot r\cdot (1-\frac{1}{(1+r)^{n+1}})}{1-\frac{1}{(1+r)^n}}$$

Suppose that $X$ is the cashflow of an investment, then a higher $A$ or $P_\infty$ implies that this investment is favorable. 

# Term Structure
The YTM of a bond with different mature dates are usually not the same. The bonds with longer mature dates usually have a higher YTM. We denote the **spot rate** $s_t$ to be the annual interest rate of money held from now, until the time $t$. The spot rate $s_t$ is usually computed from the YTM of a zero-coupon bond which mature in time t. In the term structure, we are assuming that the money held for different time will have different interest rates. Thus, if we assume yearly compounding, then when computing the present value of a cashflow, we use $d_1 = \frac{1}{1+s_1}$ to discount the payment received in year $1$, and $d_2 = \frac{1}{(1+s_2)^2}$ to discount the payment received in year 2. If we assume the continuous compounding, then $d_1 = e^{-s_1}$, $d_2 = e^{-2s_2}$. 

# Forward Rates
Since the interest rate for holding the money for 1 year is different from holding the money for 2 years. We also expect that interest rate for borrowing the money for 1 year now is different from borrowing the money for 1 year next year. We call the latter as the **forward rates**. More precisely, the forward rate $f_{i, j}$ with $j>i$ are the interest rates for money to be borrowed from time $i$ to time $j$ in the future, but under terms agreed upon today. Under the yearly compounding assumption, the forward rates can be computed from the equation
$$(1+s_j)^j = (1+s_i)^i(1+f_{i,j})^{j-i}$$
By solving this equation, we get 
$$f_{i,j} = \left(\frac{(1+s_j)^j}{(1+s_i)^i}\right)^{1/(j-i)}-1$$

If we assume continuous compounding, then we have 
$$e^{js_j} = e^{is_i}e^{(j-i)f_{i,j}}$$
which implies that 
$$js_j = is_i + (j-i)f_{i,j}$$
By solving this equation, we get 
$$f_{i,j} = \frac{js_j-is_i}{j-i}$$


**Exercise 4.1:** (One forward rate) \
If the spot rates for 1 and 2 years are $s_1$ = 6.3% and $s_2$ = 6.9%, what is the forward rate $f_{1,2}$? Assume continuous compounding for the computation. 

In [1]:
f_12 = (2 * 0.069 - 0.063)/(2-1)
print(f"The forward rate f_12 is {f_12*100:.2f}%")

The forward rate f_12 is 7.50%


**Exercise 4.2:** (Spot update) \
Given the (yearly) spot rate curve s = (5.0, 5.3, 5.6, 5.8, 6.0, 6.1), find the spot rate curve for next year. Assume continuous compounding for the computation.

In [1]:
s = [5, 5.3, 5.6, 5.8, 6.0, 6.1]
s_new = []
for i in range(1, len(s)):
    s_new.append(((i+1) * s[i] - s[0])/i)
s_new_formated= [ '%.2f' % elem for elem in s_new ]
print("The spot rates for next year is", s_new_formated)

The spot rates for next year is ['5.60', '5.90', '6.07', '6.25', '6.32']


**Exercise 4.3** (Construction of a zero coupon bond) \
Consider two 5-year bonds: one has a 9% coupon and sells for 101.00; the other has a 7% coupon and sells for 93.20. Find the price of a 5-year zero-coupon bond.

**Solution** \
Suppose we buy $x_1$ units of the 9% bond, and $x_2$ units of the 7% bond. Then in order for this portfolio to be 1 unit of the zero-coupon bond, we need 
$$\begin{cases}100x_1 + 100x_2 = 100\\
9x_1+7x_2 = 0\end{cases}$$
Solving this equation gives $x_1 = -3.5$ and $x_2 = 4.5$. Hence, the price of the 5-year zero coupon bond is
$$-3.5\cdot 101 + 4.5\cdot 93.2 = 65.9$$

**Exercise 4.5**(Instantaneous rates)\
Let s(t), $0 < t < \infty$, denote a spot rate curve; that is, the present value of a dollar to be received at time t is $e^{-s(t)t}$, For  $t_1<t_2$, let $f(t_1, t_2)$ be the forward rate between $t_1$ and $t_2$ implied by the given spot rate curve.

(a) Find an expression for $f(t_1, t_2)$.\
(b) Let $r(t) = \lim\limits_{t_2\rightarrow t}f(t, t_2)$. We can call $r(t)$ the **instantaneous interest rate** at time $t$. Show that $r(t) = s(t) +s'(t)t$. \
(c) Suppose an amount $x_0$ is invested in a bank account at t = 0 which pays the instantaneous rate of interest r(t) at all t (compounded). Then the bank balance $x(t)$ will satisfy $dx(t)/dt = r(t)x(t)$. Find an expression for x(t).

**Solution:** \
a) From the definition of forward rates, we have 
$$e^{s(t_1)t_1} = e^{s(t_2)t_2}e^{f(t_1, t_2)(t_2-t_1)}$$
By solving $f(t_1, t_2)$, we get
$$f(t_1, t_2) = \frac{t_2s(t_2)-t_1s(t_1)}{t_2-t_1}$$ 

b) Using the L'Hospital Rule, we have
$$r(t) = \lim\limits_{t_2\rightarrow t} f(t, t_2) =  \lim\limits_{t_2\rightarrow t}\frac{t_2s(t_2)-ts(t)}{t_2-t} = \lim\limits_{t_2\rightarrow t} \frac{s(t_2)+t_2s'(t_2) - 0}{1} = s(t)+ts'(t)$$


c) We have 
$$\frac{dx(t)}{x(t)} = r(t)dt = (s(t)+ts'(t))dt = d(ts(t))$$
Integrating both sides gives 
$$\ln(x(t)) = ts(t)+C$$
and we have 
$$x(t) = e^Ce^{ts(t)}$$
By substituting $t=0$, we get $e^C = x_0$, and thus, we have 
$$x(t) = x_0e^{ts(t)}$$

**Exercise 4.7** (Bond taxes) An investor is considering the purchase of 10-year U.S. Treasury bonds and plans to hold them to maturity. Federal taxes on coupons must be paid during the year they are received, and tax must also be paid on the capital gain realized at maturity (defined as the difference between face value and original price). Federal bonds are exempt from state taxes. This investor’s federal tax bracket rate is t = 30%, as it is for most individuals. There are two bonds that meet the investor’s requirements. Bond 1 is a 10-year, 10% bond with a price (in decimal form) of $P_1$ = 92.21. Bond 2 is a 10-year, 7% bond with a price of $P_2$ = 75.84. Based on the price information contained in those two bonds, the investor would like to compute the theoretical price of a hypothetical 10-year zero coupon bond that had no coupon payments and required tax payment only at maturity equal in amount to 30% of the realized capital gain (the face value minus the original price). This theoretical price should be such that the price of this bond and those of bonds 1 and 2 are mutually consistent on an after-tax basis. Find this theoretical price, and show that it does not depend on the tax rate t. (Assume all cash flows occur at the end of each year.)

**Solution**
Let $P_1, P_2$ be the price of the 10% bond and 7% bond respectively. Suppose that $P_0$ is the price of the zero coupon 10 year bond, and $t$ is the tax rate. We suppose that the zero coupon bond is constructed by using $x_1$ unit of the $10%$ bond, and $x_2$ unit of the $7%$ bond. In order to cancel the coupon payment, we must have 
$$x_1(1-t)\cdot 10 + x_2(1-t)\cdot 7 = 0$$
which is equivalent to 
$$10x_1+7x_2 = 0$$
We also want the after tax payment at expiry to match, hence, we have 
$$x_1(100-(100-P_1)t) + x_2(100 - (100-P_2)t) = 100-(100-P_0)t$$
The price of this zero coupon bond satisfies $P_0 = x_1P_1+x_2P_2$. By substituting this to the above equality, we get 
$$x_1+x_2 = 1$$
Hence, the $x_1, x_2$ satisfies the equation 
$$\begin{cases}10x_1+7x_2 = 0 \\ x_1+x_2 = 1\end{cases}$$
We solve that $x_1 = -7/3, x_2 = 10/3$, and the price of the zero coupon bond is $x_1P_1+x_2P_2 = 37.64$, which does not depends on the tax rate $t$. 